# Home Deco Business Analysis - Statistical Tests

This notebook contains statistical analysis of the Home Deco business data, including:
1. Correlation Analysis
2. Hypothesis Testing
3. A/B Testing (if applicable)
4. Distribution Analysis

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# Set style for all visualizations
plt.style.use('seaborn-whitegrid')
sns.set_palette('Greys')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
# Load the datasets
daily_clients = pd.read_excel('../data/daily_clients.xlsx')
monthly_sales = pd.read_excel('../data/monthly_sales_ads_data.xlsx')

## 1. Correlation Analysis
Analyzing relationships between sales, web traffic, and marketing metrics